#Motas sensoras aula 3304
Vanessa Rodríguez Horcajo - TFM

En este notebook se recogen todas las funciones definidas y pruebas ejecutadas para el entendimiento y tratamiento de las mediciones recogidas por las motas sensoras colocadas en el aula 3304.

Se disponen de mediciones desde noviembre de 2023 hasta marzo de 2024. Para facilitar el procesamiento de las mediciones recogidas por las motas del aula 3304, todos los datos a procesar se encuentran almacenados en Google Drive y será necesario realizar un filtrado por los meses y motas de interés.

In [ ]:
import os
from google.colab import drive
import pandas as pd
from datetime import datetime

Acceso a Google Drive para la carga de datos

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Entendimiento y procesamiento de los datos


Antes de comenzar con el procesamiento de los datos, es importante conocer qué datos se almacenan y qué estructura presentan, así como determinar qué cambios es necesario realizar para conseguir unos datos limpios y estructurados con los que trabajar posteriormente.


In [ ]:
r_3304 = '/content/drive/My Drive/TFM/Motas3304/Motas3304.csv'

df_3304 = pd.read_csv(r_3304, sep=',', encoding='utf-8', dtype={'_id.oid':str})
df_3304.head(3)

,_id.oid,time,class,hub,node,data.humidity,data.room_temp,data.luminosity,data.add_temp,data.surf_temp,_id,data.movement,data.noise,data.co2
0,62b5f6bbcef811c5081a1392,2022-06-24T19:39:07.823156,4405,000FF001,131332,33.6,24.7,230.83,24.73,23.21,NaN,NaN,NaN,NaN
1,NaN,2023-04-21T10:03:53.838274,4405,000FF001,65794,29.2,26.9,303.33,26.03,22.83,6442436986e2ed13ca9299b4,NaN,NaN,NaN
2,NaN,2023-04-21T10:04:07.796869,4405,000FF001,65794,NaN,NaN,NaN,NaN,NaN,6442437786e2ed13ca9299b5,0.0,NaN,NaN


In [ ]:
df_3304.shape

(240798, 14)

Se disponen de 240798 muestras en total. Sin embargo, tal y como puede observarse en la columna time, muchas de estas muestras pertenecen a meses que nos son de interés en el análisis. Por ese motivo, en primer lugar, se realiza un filtrado de los datos por los meses de noviembre a marzo.

In [ ]:
type(df_3304['time'][0])

str

In [ ]:
df_3304['time'] = pd.to_datetime(df_3304['time'])
type(df_3304['time'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
meses = [11, 12, 1, 2, 3]
years = [2023, 2024]
df_3304 = df_3304[df_3304['time'].dt.month.isin(meses) & df_3304['time'].dt.year.isin(years)]
df_3304.reset_index(drop=True, inplace=True)
df_3304.head(3)

,_id.oid,time,class,hub,node,data.humidity,data.room_temp,data.luminosity,data.add_temp,data.surf_temp,_id,data.movement,data.noise,data.co2
0,NaN,2023-11-03 08:53:29.378251,4405,000FF001,65793,47.1,20.6,73.33,19.83,20.77,6544a6f957e6e13d76f204ac,NaN,NaN,NaN
1,NaN,2023-11-03 08:53:31.348655,4405,000FF001,131332,49.3,18.2,230.00,17.47,17.47,6544a6fb57e6e13d76f204ad,NaN,NaN,NaN
2,NaN,2023-11-03 08:54:30.234509,4405,000FF001,65794,NaN,NaN,NaN,NaN,NaN,6544a73657e6e13d76f204ae,0.0,NaN,NaN


In [ ]:
df_3304.shape

(75021, 14)

Tas el filtrado de los datos, el número de muestras queda reducido a 75021 para los meses de noviembre a marzo.

Antes de comenzar con el entendimiento de los datos, se eliminan las columnas que no contienen información relevante en relación con el uso que se pretende dar a estos datos.

In [ ]:
df_3304 = df_3304.drop(['_id.oid', 'class', 'hub', '_id', 'data.add_temp', 'data.surf_temp'], axis=1)
df_3304.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement,data.noise,data.co2
0,2023-11-03 08:53:29.378251,65793,47.1,20.6,73.33,NaN,NaN,NaN
1,2023-11-03 08:53:31.348655,131332,49.3,18.2,230.00,NaN,NaN,NaN
2,2023-11-03 08:54:30.234509,65794,NaN,NaN,NaN,0.0,NaN,NaN


Tal y como puede observarse, las columnas de ruido y CO2 se encuentran completamente vacías. De esta manera, se eliminan también del conjunto de datos.

In [ ]:
df_3304['data.noise'].unique()

array([nan])

In [ ]:
df_3304['data.co2'].unique()

array([nan])

In [ ]:
df_3304 = df_3304.drop(['data.noise', 'data.co2'], axis=1)
df_3304.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:53:29.378251,65793,47.1,20.6,73.33,NaN
1,2023-11-03 08:53:31.348655,131332,49.3,18.2,230.00,NaN
2,2023-11-03 08:54:30.234509,65794,NaN,NaN,NaN,0.0


Una vez finalizada la limpieza preliminar de los datos, es momento de entender qué datos se almacenan en el conjunto de datos de entrada. Este conjunto de datos, tal y como es posible observar en la celda anterior, presenta la siguiente estructura:
  * `time`: hora de recogida de la medición.
  * `node`: identificador de la mota sensora que recogió la medición.
  * `data.humidity`: valor de humedad recogido por la mota.
  * `data.room_temp`: valor de temperatura recogido por la mota.
  * `data.luminosity`: valor de luminosidad recogido por la mota.
  * `data.movement`: valor de movimiento recogido por la mota. Devuelve un 0.0 cuando detecta movimiento.

Una vez establecida esta caracterización, es posible destacar lo siguiente:
  * Existen tres motas con identificadores diferentes (65793, 131332, 65794) en la recopilación de datos para los meses de noviembre a marzo. Cada una de ellas se encarga de la recogida de un grupo de magnitudes diferente. Es necesario determinar qué magnitudes se encarga de medir cada una de las motas.
  * La magnitud de detección de movimiento es independiente del resto de magnitudes, es decir, cuando una mota detecta movimiento, envía una trama vacía a excepción de la posición correspondiente al valor de movimiento que contiene un 0.0, tal y como puede observarse en la fila 2 de la celda anterior.
  * A excepción de las magnitudes de ruido y CO2 para las que no se dispone de información, el resto de magnitudes recogidas se corresponden 1 a 1 con las magnitudes recogidas por las motas del aula 3301.



In [ ]:
df_3304['node'].unique()

array([ 65793, 131332,  65794])

Para facilitar la identificación de las magnitudes medidas por cada una de las motas, se dividen los datos originales en tres partes, una por cada mota sensora.

In [ ]:
type(df_3304['node'][0])

numpy.int64

In [ ]:
df_3304_65793 = df_3304[(df_3304['node'] == 65793)]
df_3304_131332 = df_3304[(df_3304['node'] == 131332)]
df_3304_65794 = df_3304[(df_3304['node'] == 65794)]

df_3304_65793.reset_index(drop=True, inplace=True)
df_3304_131332.reset_index(drop=True, inplace=True)
df_3304_65794.reset_index(drop=True, inplace=True)

### Mota 65793

Se comienza analizando los datos recogidos por la mota con identificador 65793. Tal y como puede observarse, esta mota se encarga de la recopilación de datos para las magnitudes de humedad, temperatura y luminosidad. La columna de detección de movimiento se encuentra completamente vacía. El resto de muestras no contiene ningún valor nulo. La periodicidad de las mediciones es de 5 minutos.

In [ ]:
df_3304_65793.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:53:29.378251,65793,47.1,20.6,73.33,NaN
1,2023-11-03 08:58:33.447027,65793,47.5,20.9,71.67,NaN
2,2023-11-03 09:03:37.505871,65793,47.8,20.8,70.83,NaN


In [ ]:
filas_vacias = df_3304_65793[df_3304_65793.isnull().all(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
filas_vacias = df_3304_65793[df_3304_65793.iloc[ : ,2:5].isnull().any(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
df_3304_65793['data.movement'].unique()

array([nan])

### Mota 131332

A continuación se analizan los datos recogidos por la mota con identificador 131332. Tal y como puede observarse, al igual que la mota con identificador 65793, esta mota se encarga de la recopilación de datos para las magnitudes de humedad, temperatura y luminosidad. La columna de detección de movimiento se encuentra completamente vacía. El resto de muestras no contiene ningún valor nulo. La periodicidad de las mediciones es de 5 minutos.

In [ ]:
df_3304_131332.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:53:31.348655,131332,49.3,18.2,230.00,NaN
1,2023-11-03 08:58:35.181794,131332,49.3,18.3,211.67,NaN
2,2023-11-03 09:03:39.054281,131332,49.6,18.4,216.67,NaN


In [ ]:
filas_vacias = df_3304_131332[df_3304_131332.isnull().all(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
filas_vacias = df_3304_131332[df_3304_131332.iloc[ : ,2:5].isnull().any(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
df_3304_131332['data.movement'].unique()

array([nan])

### Mota 64794

Para finalizar, se analizan los datos recogidos por la mota con identificador 65794. Tal y como puede observarse, a diferencia de las dos motas analizadas previamente, esta mota se encarga de la recopilación de datos para las magnitudes de humedad, temperatura y luminosidad así como de la detección de movimiento. Existen muestras dedicadas únicamente a la transmisión de las magnitudes físicas y otras dedicadas únicamente a la transmisión de la detección de movimiento.

In [ ]:
df_3304_65794.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:54:30.234509,65794,NaN,NaN,NaN,0.0
1,2023-11-03 08:55:06.294222,65794,55.1,17.2,170.83,NaN
2,2023-11-03 08:56:02.264852,65794,NaN,NaN,NaN,0.0


In [ ]:
filas_vacias = df_3304_65794[df_3304_65794.isnull().all(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
filas_vacias = df_3304_65794[df_3304_65794.iloc[ : ,2:5].isnull().any(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:54:30.234509,65794,NaN,NaN,NaN,0.0
2,2023-11-03 08:56:02.264852,65794,NaN,NaN,NaN,0.0
3,2023-11-03 08:58:36.299210,65794,NaN,NaN,NaN,0.0
4,2023-11-03 08:59:46.313674,65794,NaN,NaN,NaN,0.0
6,2023-11-03 09:01:18.343700,65794,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
29894,2024-03-15 14:36:07.778111,65794,NaN,NaN,NaN,0.0
29895,2024-03-15 14:37:31.803582,65794,NaN,NaN,NaN,0.0
29897,2024-03-15 14:38:07.821791,65794,NaN,NaN,NaN,0.0
29910,2024-03-15 15:41:49.035923,65794,NaN,NaN,NaN,0.0


In [ ]:
df_3304_65794['data.movement'].unique()

array([ 0., nan])

In [ ]:
df_3304_65794_m = df_3304_65794[(df_3304_65794['data.movement'] == 0.0)]
df_3304_65794 = df_3304_65794[(df_3304_65794['data.movement'].isnull())]

df_3304_65794_m.reset_index(drop=True, inplace=True)
df_3304_65794.reset_index(drop=True, inplace=True)

Tal y como puede observarse, tras la diferenciación de las muestras de detección de movimiento y medición de las magnitudes físicas en grupos diferentes, 14492 muestras son exclusivamente de la detección de movimiento. Puesto que no aportan valor a los datos de entrada con el objetivo de uso que se pretende dar a los datos, estas muestras se eliminan del conjunto de datos. De esta manera, los datos restantes de esta mota presentan las mismas caracerísticas que los conjuntos de datos de las motas anteriores: contiene la recopilación de datos para las magnitudes de humedad, temperatura y luminosidad, la columna de detección de movimiento se encuentra completamente vacía, el resto de muestras no contiene ningún valor nulo y la periodicidad de las mediciones es de 5 minutos.

In [ ]:
df_3304_65794_m.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:54:30.234509,65794,NaN,NaN,NaN,0.0
1,2023-11-03 08:56:02.264852,65794,NaN,NaN,NaN,0.0
2,2023-11-03 08:58:36.299210,65794,NaN,NaN,NaN,0.0


In [ ]:
df_3304_65794_m.shape

(14492, 6)

In [ ]:
df_3304_65794.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:55:06.294222,65794,55.1,17.2,170.83,NaN
1,2023-11-03 09:00:08.373296,65794,55.2,17.4,169.17,NaN
2,2023-11-03 09:05:10.447407,65794,55.6,17.4,175.83,NaN


In [ ]:
filas_vacias = df_3304_65794[df_3304_65794.iloc[ : ,2:5].isnull().any(axis=1)]
filas_vacias

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
df_3304_65794['data.movement'].unique()

array([nan])

In [ ]:
df_3304_65794.shape

(15791, 6)

### Unificación y almacenamiento de los datos

Tras el análisis anterior y la identificación de las muestras de interés, se unifican los distintos conjuntos de muestras para obtener el conjunto de datos final con el que seguir trabajando. Este conjunto de datos contendrá las muestras recogidas por las dos primeras motas analizadas así como las muestras recogidas por la última mota relativas a la medición de las magnitudes de temperatura, humedad y luminosidad. Puesto que la columna de movimiento se encuentra completamente vacía, se elimina del conjunto de datos.

In [ ]:
df_3304_65793['data.movement'].unique()

array([nan])

In [ ]:
df_3304_131332['data.movement'].unique()

array([nan])

In [ ]:
df_3304_65794['data.movement'].unique()

array([nan])

In [ ]:
df_motas3304 = pd.concat([df_3304_65793, df_3304_131332, df_3304_65794], ignore_index=True)

In [ ]:
df_motas3304.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement
0,2023-11-03 08:53:29.378251,65793,47.1,20.6,73.33,NaN
1,2023-11-03 08:58:33.447027,65793,47.5,20.9,71.67,NaN
2,2023-11-03 09:03:37.505871,65793,47.8,20.8,70.83,NaN


In [ ]:
df_3304_65794_m = df_3304_65794[(df_3304_65794['data.movement'] == 0.0)]
df_3304_65794_m

,time,node,data.humidity,data.room_temp,data.luminosity,data.movement


In [ ]:
df_motas3304.shape

(60529, 6)

In [ ]:
df_motas3304['data.movement'].unique()

array([nan])

In [ ]:
df_motas3304 = df_motas3304.drop(['data.movement'], axis=1)
df_motas3304.head(3)

,time,node,data.humidity,data.room_temp,data.luminosity
0,2023-11-03 08:53:29.378251,65793,47.1,20.6,73.33
1,2023-11-03 08:58:33.447027,65793,47.5,20.9,71.67
2,2023-11-03 09:03:37.505871,65793,47.8,20.8,70.83


In [ ]:
df_motas3304['Date'] = df_motas3304['time'].dt.date
df_motas3304['Date'] = df_motas3304['Date'].apply(lambda x: x.strftime('%d/%m/%Y'))
df_motas3304['Time'] = df_motas3304['time'].dt.time
df_motas3304['Time'] = df_motas3304['Time'].apply(lambda x: x.strftime('%H:%M'))
df_motas3304 = df_motas3304.drop(['time'], axis=1)
df_motas3304.head(3)

,node,data.humidity,data.room_temp,data.luminosity,Date,Time
0,65793,47.1,20.6,73.33,03/11/2023,08:53
1,65793,47.5,20.9,71.67,03/11/2023,08:58
2,65793,47.8,20.8,70.83,03/11/2023,09:03


In [ ]:
nuevos_nombres = {'data.humidity': 'Humidity',
                  'data.room_temp': 'Temperature',
                  'data.luminosity': 'Luminosity',
                  'node': 'Mota'
                  }

df_motas3304 = df_motas3304.rename(columns=nuevos_nombres)
df_motas3304.head(1)

,Mota,Humidity,Temperature,Luminosity,Date,Time
0,65793,47.1,20.6,73.33,03/11/2023,08:53


In [ ]:
df_motas3304 = df_motas3304[['Date', 'Time', 'Temperature', 'Humidity', 'Luminosity', 'Mota']]
df_motas3304.head(1)

,Date,Time,Temperature,Humidity,Luminosity,Mota
0,03/11/2023,08:53,20.6,47.1,73.33,65793


Para unificar la nomenclatura de los códigos de las motas sensoras de ambas aulas, los códigos de las motas del aula 3304, se mapearán de la siguiente forma:
 * M1 : nodo 65794
 * M2 : nodo 131332
 * M3 : nodo 65793

In [ ]:
def mapeo_indices(indice):
  mota = 'M'
  if indice == 65794:
    mota = 'M1'
  elif indice == 131332:
    mota = 'M2'
  else:
    mota = 'M3'

  return mota

In [ ]:
df_motas3304['Mota'] = df_motas3304['Mota'].apply(mapeo_indices)

In [ ]:
df_motas3304 = df_motas3304[['Date', 'Time', 'Temperature', 'Humidity', 'Luminosity', 'Mota']]
df_motas3304.head(1)

,Date,Time,Temperature,Humidity,Luminosity,Mota
0,03/11/2023,08:53,20.6,47.1,73.33,M3


In [ ]:
df_motas3304.Mota.unique()

array(['M3', 'M2', 'M1'], dtype=object)

In [ ]:
df_motas3304.to_csv('/content/drive/My Drive/TFM/Datos_Finales/Motas3304_data.csv', index=False, sep=';')